In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
# Add the project root (toy-wm directory) to sys.path so we can import src as a package
# This allows relative imports within the package to work correctly
# Assumes notebook is run from the project root directory
project_root = os.getcwd()  # or use: os.path.dirname(os.path.abspath(''))
print(project_root)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

/share/u/wendler/code/toy-wm


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch as t

# Import using absolute path from src package
from src.utils.checkpoint import load_model_from_config

#model = load_model_from_config("configs/small_6frame.yaml")
model = load_model_from_config("configs/bigger_30frame_causal.yaml", strict=False)
model.eval()

loading configs/bigger_30frame_causal.yaml


CausalDit(
  (rope_seq): RoPE()
  (blocks): ModuleList(
    (0-7): 8 x CausalBlock(
      (norm1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      (selfattn): AttentionEinOps(
        (rope): RoPE()
        (ln1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (ln2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      )
      (norm2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      (geglu): GEGLU(
        (up_proj): Linear(in_features=384, out_features=1536, bias=True)
        (up_gate): Linear(in_features=384, out_features=1536, bias=True)
        (down): Linear(in_features=1536, out_features=384, bias=True)
        (nonlin): SiLU()
      )
      (modulation): Sequential(
        (0): SiLU()
        (1): Linear(in_features=384, out_features=2304, bias=True)
      )
    )
  )
  (patch): Patch(
    (init_conv_seq): Sequential(
      (0): Conv2d(3, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (1): SiLU()
      (2): GroupNorm(3

In [3]:
model.activate_caching(1, 300)

torch.Size([1, 19500, 1, 32]) torch.Size([1, 19500, 1, 32])
updating rope for block 0
torch.Size([1, 1950, 1, 32]) torch.Size([1, 1950, 1, 32])
torch.Size([1, 19500, 1, 32]) torch.Size([1, 19500, 1, 32])
updating rope for block 1
torch.Size([1, 1950, 1, 32]) torch.Size([1, 1950, 1, 32])
torch.Size([1, 19500, 1, 32]) torch.Size([1, 19500, 1, 32])
updating rope for block 2
torch.Size([1, 1950, 1, 32]) torch.Size([1, 1950, 1, 32])
torch.Size([1, 19500, 1, 32]) torch.Size([1, 19500, 1, 32])
updating rope for block 3
torch.Size([1, 1950, 1, 32]) torch.Size([1, 1950, 1, 32])
torch.Size([1, 19500, 1, 32]) torch.Size([1, 19500, 1, 32])
updating rope for block 4
torch.Size([1, 1950, 1, 32]) torch.Size([1, 1950, 1, 32])
torch.Size([1, 19500, 1, 32]) torch.Size([1, 19500, 1, 32])
updating rope for block 5
torch.Size([1, 1950, 1, 32]) torch.Size([1, 1950, 1, 32])
torch.Size([1, 19500, 1, 32]) torch.Size([1, 19500, 1, 32])
updating rope for block 6
torch.Size([1, 1950, 1, 32]) torch.Size([1, 1950, 

In [4]:
from src.datasets.pong1m import get_loader

loader, pred2frame = get_loader(duration=1, fps=30, mode='-1,1')
frames, actions = next(iter(loader))
frames = pred2frame(frames)

520 batches


In [ ]:
from src.trainers.diffusion_forcing import sample

def render_video(actions, n_steps=8, cfg=0, negative_actions=None, clamp=True):
    device = model.device
    frames = t.randn(1, actions.shape[1], 3, 24, 24, device="cpu")
    for aidx in range(actions.shape[1]):
        print(aidx)
        noise = t.randn(1, 1, 3, 24, 24, device=device)
        z = sample(model, noise, actions[:, aidx:aidx+1], num_steps=n_steps, cfg=cfg, negative_actions=negative_actions)
        model.cache.update_global_location(1)
        frames[:, aidx:aidx+1] = z.detach().cpu()
        if clamp:
            frames = frames.clamp(-1, 1)
    return frames

In [6]:
model = t.compile(model)

In [7]:
actions = t.tensor([[0]+119*[2]], dtype=t.int32, device=model.device)
negative_actions = t.tensor([[0]+29*[2]], dtype=t.int32, device=model.device)
z = render_video(actions, n_steps=4, cfg=0, negative_actions=None, clamp=True)
frames = pred2frame(z)

import matplotlib.pyplot as plt
import matplotlib.animation as animation

# z[0] shape: (num_frames, height, width, 3), where num_frames = frames.shape[1]
frames_np = frames[0].permute(0, 2, 3, 1).cpu().numpy()  # shape: (num_frames, height, width, 3)

fig, ax = plt.subplots()
im = ax.imshow(frames_np[0])
ax.set_title('Frame 0')

def animate(i):
    im.set_data(frames_np[i])
    ax.set_title(f'Frame {i}')
    return [im]

ani = animation.FuncAnimation(
    fig, animate, frames=frames_np.shape[0],
    interval=200, blit=True, repeat=True
)

plt.show()

# Optional: To save the gif uncomment below (requires pillow, install with `pip install pillow`)
from collections import Counter
# Majority vote over values in actions for the filename
if actions is not None:
    # Flatten actions tensor and convert to list
    action_vals = actions.flatten().tolist()
    majority_val = Counter(action_vals).most_common(1)[0][0]
    ani.save(f"sample_{majority_val}.gif", writer="pillow")
else:
    ani.save("sample_unknown.gif", writer="pillow")




0


  0%|          | 0/4 [00:00<?, ?it/s]

RoPE forward torch.Size([1, 65, 12, 32]) 0
RoPE forward torch.Size([1, 65, 12, 32]) 0
RoPE forward torch.Size([1, 65, 12, 32]) 0
RoPE forward torch.Size([1, 65, 12, 32]) 0
RoPE forward torch.Size([1, 65, 12, 32]) 0
RoPE forward torch.Size([1, 65, 12, 32]) 0
RoPE forward torch.Size([1, 65, 12, 32]) 0
RoPE forward torch.Size([1, 65, 12, 32]) 0
RoPE forward torch.Size([1, 65, 12, 32]) 0
RoPE forward torch.Size([1, 65, 12, 32]) 0
RoPE forward torch.Size([1, 65, 12, 32]) 0
RoPE forward torch.Size([1, 65, 12, 32]) 0
RoPE forward torch.Size([1, 65, 12, 32]) 0
RoPE forward torch.Size([1, 65, 12, 32]) 0
RoPE forward torch.Size([1, 65, 12, 32]) 0
RoPE forward torch.Size([1, 65, 12, 32]) 0


 25%|██▌       | 1/4 [00:41<02:03, 41.30s/it]W1105 16:16:16.429000 105160 .venv/lib/python3.13/site-packages/torch/_dynamo/convert_frame.py:1358] [4/8] torch._dynamo hit config.recompile_limit (8)
W1105 16:16:16.429000 105160 .venv/lib/python3.13/site-packages/torch/_dynamo/convert_frame.py:1358] [4/8]    function: 'forward' (/share/u/wendler/code/toy-wm/src/models/dit_dforce.py:39)
W1105 16:16:16.429000 105160 .venv/lib/python3.13/site-packages/torch/_dynamo/convert_frame.py:1358] [4/8]    last reason: 4/7: self.layer_idx == 7                                      # assert layer_idx == self.curr_layer, f"layer idx should be the same as our internal counter but we got {layer_idx} and internal is {self.curr_layer}."  # src/nn/attn.py:31 in get (HINT: torch.compile considers integer attributes of the nn.Module to be static. If you are observing recompilation, you might want to make this integer dynamic using torch._dynamo.config.allow_unspec_int_on_nn_module = True, or convert this intege

RoPE forward torch.Size([1, 65, 12, 32]) 65
RoPE forward torch.Size([1, 65, 12, 32]) 65
RoPE forward torch.Size([1, 65, 12, 32]) 65
RoPE forward torch.Size([1, 65, 12, 32]) 65
RoPE forward torch.Size([1, 65, 12, 32]) 65
RoPE forward torch.Size([1, 65, 12, 32]) 65
RoPE forward torch.Size([1, 65, 12, 32]) 65
RoPE forward torch.Size([1, 65, 12, 32]) 65
RoPE forward torch.Size([1, 65, 12, 32]) 65
RoPE forward torch.Size([1, 65, 12, 32]) 65
RoPE forward torch.Size([1, 65, 12, 32]) 65
RoPE forward torch.Size([1, 65, 12, 32]) 65
RoPE forward torch.Size([1, 65, 12, 32]) 65
RoPE forward torch.Size([1, 65, 12, 32]) 65
RoPE forward torch.Size([1, 65, 12, 32]) 65
RoPE forward torch.Size([1, 65, 12, 32]) 65
RoPE forward torch.Size([1, 65, 12, 32]) 130
RoPE forward torch.Size([1, 65, 12, 32]) 130
RoPE forward torch.Size([1, 65, 12, 32]) 130
RoPE forward torch.Size([1, 65, 12, 32]) 130
RoPE forward torch.Size([1, 65, 12, 32]) 130
RoPE forward torch.Size([1, 65, 12, 32]) 130
RoPE forward torch.Size([1

...

100%|██████████| 4/4 [00:00<00:00, 20.72it/s]


RoPE forward torch.Size([1, 65, 12, 32]) 19435
RoPE forward torch.Size([1, 65, 12, 32]) 19435
75


  0%|          | 0/4 [00:00<?, ?it/s]


RoPE forward torch.Size([1, 65, 12, 32]) 19500


RuntimeError: The size of tensor a (0) must match the size of tensor b (65) at non-singleton dimension 1

In [ ]:
actions = t.tensor([6*[2]], dtype=t.int32, device="mps")
frames = render_video(actions, z[:, 0])
frames = pred2frame(frames)

import matplotlib.pyplot as plt
import matplotlib.animation as animation

# z[0] shape: (num_frames, height, width, 3), where num_frames = frames.shape[1]
frames_np = frames[0].permute(0, 2, 3, 1).cpu().numpy()  # shape: (num_frames, height, width, 3)

fig, ax = plt.subplots()
im = ax.imshow(frames_np[0])
ax.set_title('Frame 0')

def animate(i):
    im.set_data(frames_np[i])
    ax.set_title(f'Frame {i}')
    return [im]

ani = animation.FuncAnimation(
    fig, animate, frames=frames_np.shape[0],
    interval=200, blit=True, repeat=True
)

plt.show()

# Optional: To save the gif uncomment below (requires pillow, install with `pip install pillow`)
ani.save("sample_cont.gif", writer="pillow")